In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image
from os import listdir

In [58]:
import glob
import PIL
# normalise the data for faster training
from sklearn.preprocessing import StandardScaler

In [53]:
parasite_files = glob.glob('Parasitized/*.png')

In [103]:
parasite_pic = image.imread('Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png')

In [104]:
print(parasite_pic.dtype)

float32


In [105]:
print(parasite_pic.shape)

(148, 142, 3)


In [54]:
uninfected_files = glob.glob('Uninfected/*.png')

In [84]:
SS = StandardScaler()

In [110]:
# convert the image from 3 channels to 1-channel
from skimage.color import rgb2gray, gray2rgb

gray_pic = rgb2gray(parasite_pic)

In [108]:
from sklearn.preprocessing import normalize

In [111]:
normalize(gray_pic, axis=0)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [114]:
print(gray_pic.shape)

(148, 142)


In [116]:
X_parasite = []
Y_parasite = []

In [117]:
for file in parasite_files:
    image_data = image.imread(file)
    image_data = rgb2gray(image_data)
    image_data = normalize(image_data)
    X_parasite.append(image_data)
    Y_parasite.append(1)
    # append 1 to Y as target infected variable

In [118]:
X_uninfected = []
Y_uninfected = []

In [119]:
for file in uninfected_files:
    image_data2 = image.imread(file)
    image_data2 = rgb2gray(image_data)
    image_data2 = normalize(image_data)
    X_uninfected.append(image_data2)
    Y_uninfected.append(0)
    # appent 0 to Y as target not infected

In [91]:
print(image_data2.dtype)

float32


In [120]:
# now to combine the images with their target variables
X = X_parasite + X_uninfected
Y = Y_parasite + Y_uninfected

In [121]:
# need to shuffle the data so its not biased
from sklearn.utils import shuffle
X, Y = shuffle(X, Y)

In [122]:
# split into training and test data
from sklearn.model_selection import train_test_split

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=42)

In [124]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn import linear_model

In [125]:
logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1, multi_class='multinomial')
rbm = BernoulliRBM(random_state=0, verbose=True)
rbm_classifier = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])

In [129]:
X_train[1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [131]:
rbm_classifier.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [101]:
#can't feed in a sequence of numbers inside the array as an element (array with an array) wont work in this case.